In [1]:
""" Improving Anomaly Detection in Audio by Learning How it Flows in Time - Yishai Shor
           """

#########################################
########## Import Libraries #############
#########################################
import os
import matplotlib.pyplot as plt
import math
import numpy as np
import sklearn
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV

# for loading and visualizing audio files
import librosa
import librosa.display
import wave, os, glob

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv3D, Conv2D, MaxPooling3D, MaxPooling2D, Dropout, BatchNormalization, \
    Reshape, Input, InputLayer, GlobalAveragePooling1D, GlobalAveragePooling3D
import tensorflow
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.activations import sigmoid
import matplotlib.pyplot as plt


#########################################
########## Defining functions ###########
#########################################

def load_audio(kind):
    # Step 1: Define path
    if kind == 'train':
        audio_fpath = "/content/google_colab/train1"
        audio_clips = os.listdir(audio_fpath)
        zero = []
        for i in range(0, len(audio_clips)):
            x, sr = librosa.load(audio_fpath + '/' + audio_clips[i], sr=41000)
            zero.append(x)
    elif kind == 'test':
        audio_fpath = "/content/google_colab/test_normal1"
        audio_clips = os.listdir(audio_fpath)
        zero = []
        for i in range(0, len(audio_clips)):
            x, sr = librosa.load(audio_fpath + '/' + audio_clips[i], sr=41000)
            zero.append(x)
        audio_fpath = "/content/google_colab/test_anomaly1"
        audio_clips = os.listdir(audio_fpath)
        for i in range(0, len(audio_clips)):
            x, sr = librosa.load(audio_fpath + '/' + audio_clips[i], sr=41000)
            zero.append(x)
    # Step 2: Convert the audio waveform to spectrogram
    Xdb = []
    for i in range(0,len(zero)):
        X = librosa.stft(zero[i])
        Xdb.append(librosa.amplitude_to_db(abs(X)))
        print(i)
    return Xdb

  
def create_regular_x(single_x):
    groups = int(len(single_x)/41) # 25 groups
    X_temp = [0]*(groups)
    for i in range(0,groups,1):
        X_temp[i] = single_x[groups*(i):groups*(i+1)]
    X_run_1 = [0]*(groups-4)
    for i in range(2,groups-2,1):
        X_run_1[i-2] = [X_temp[i-2],X_temp[i-1],X_temp[i],X_temp[i+1],X_temp[i+2]]
    return X_run_1

def create_twice_fast_x (single_x):
    groups = int(len(single_x)/41) # 25 groups
    X_temp = [0]*(groups)
    for i in range(0,groups,1):
        X_temp[i-2] = single_x[groups*(i):groups*(i+1)]
    #print(len(X_temp))
    X_run_2 = [0]*(groups-8)
    for i in range(4,groups-4,1):
        X_run_2[i-4] = [X_temp[i-4],X_temp[i-1],X_temp[i],X_temp[i+1],X_temp[i+4]]
    return X_run_2

def create_oposite_order_x(single_x):
    groups = int(len(single_x)/41) # 25 groups
    X_temp = [0]*(groups)
    for i in range(0,groups,1):
        X_temp[i-2] = single_x[groups*(i):groups*(i+1)]
    X_run_3 = [0]*(groups-4)
    for i in range(2,groups-2,1):
        X_run_3[i-2] = [X_temp[i+2],X_temp[i+1],X_temp[i],X_temp[i-1],X_temp[i-2]]
    return X_run_3

def create_without_i(single_x):
    groups = int(len(single_x)/41) # 25 groups
    X_temp = [0]*(groups)
    for i in range(0,groups,1):
        X_temp[i] = single_x[groups*(i):groups*(i+1)]
    X_run_1 = [0]*(groups-4)
    Y_run_1 = [0]*(groups-4)
    for i in range(2,groups-2,1):
        X_run_1[i-2] = [X_temp[i-2],X_temp[i-1],X_temp[i+1],X_temp[i+2]]
        Y_run_1[i-2] = X_temp[i]
    return X_run_1, Y_run_1

def create_lables(length, label_kind):
    return length*[label_kind]

def create_data_for_run(model_num, kind):
    X_train = load_audio(kind)
    X_train_regular = []
    X_train_twice_fast = []
    X_train_oposite_order = []
    X_train_without_i = []
    Y_train_without_i = []
    for i in range(0,len(X_train)):
        X_train_regular.append(create_regular_x(X_train[i]))
        X_train_twice_fast.append(create_twice_fast_x(X_train[i]))
        X_train_oposite_order.append(create_oposite_order_x(X_train[i]))
        x,y = create_without_i(X_train[i])
        X_train_without_i.append(x)
        Y_train_without_i.append(y)
    x_train = []
    y_train = []
    count_regular = 0
    count_revrse = 0
    if model_num != 3 and kind == 'train':
        if model_num == 1:
            x_validate = X_train_oposite_order
        elif model_num == 2:
            x_validate = X_train_twice_fast
        for i in range(0,len(X_train_regular)):
            for j in range(0,len(X_train_regular[i])):
                x_train.append(X_train_regular[i][j])
                count_regular += 1
        for i in range(0,len(x_validate)):
            for j in range(0,len(x_validate[i])):
                x_train.append(x_validate[i][j])
                count_revrse += 1
        y_train = create_lables(count_regular,1)
        y_temp = create_lables(count_revrse,0)
        y_train = y_train[:] + y_temp[:]
    elif model_num == 3 and kind == 'train':
        for i in range(0,len(X_train_without_i)):
            for j in range(0,len(X_train_without_i[i])):
                x_train.append(X_train_without_i[i][j])
                y_train.append(Y_train_without_i[i][j])
    elif kind == 'test' and model_num != 3:
        for i in range(0,len(X_train_regular)):
            for j in range(0,len(X_train_regular[i])):
                x_train.append(X_train_regular[i][j])
                count_regular += 1
        count_regular = int(count_regular/2)
        y_train = create_lables(count_regular,1) # test normal and abnormal groups are of the same size and even
        y_temp = create_lables(count_regular,0)
        y_train = y_train[:] + y_temp[:]
    elif kind == 'test' and model_num == 3:
        for i in range(0,len(X_train_without_i)):
            for j in range(0,len(X_train_without_i[i])):
                x_train.append(X_train_without_i[i][j])
                y_train.append(Y_train_without_i[i][j])
    x_train = np.asarray(x_train).astype('float32')
    y_train = np.asarray(y_train).astype('float32')
    del X_train_regular, X_train_twice_fast, X_train_oposite_order, X_train, Y_train_without_i, X_train_without_i
    return x_train, y_train

def anomaly_decision(prediction):
    # stracture data issue
    classify_value = []
    model_prediction = []
    for i in range(0,len(prediction)):
        classify_value.append(prediction[i][0])
    # anomaly decision
    for i in range(0,len(classify_value)-20, 21):
        temp = classify_value[i:i+21]
        min_val_1 = sorted(temp)[0]
        min_val_2 = sorted(temp)[1]
        min_val_3 = sorted(temp)[2]
        if (min_val_1+min_val_2+min_val_3)< 0.1:
            model_prediction.append(0)
        else:
            model_prediction.append(1)
    return model_prediction

def anomaly_decision_model_3(mse_prediction,mae_train = 10.6, sd_train = 0.2722 ,limit = 2):
    decision = []
    for i in range(0,len(mse_prediction)):
        Z = abs(mse_prediction[i] - mae_train)/sd_train
        if Z < limit:
            decision.append(1)
        else:
            decision.append(0)
    return decision

def test_reduce_size(new_list):
    model_prediction = []
    for i in range(0,len(new_list)-20, 21):
        temp = round(min(new_list[i:i+21]))
        model_prediction.append(temp)
    return model_prediction

def accuracy(list1, list2):
    sum1 = 0
    for i in range(0,len(list1)):
        sum1 += abs(list1[i]-list2[i])
    return 1 -sum1/len(list1)

def compute_mae(list1,list2):
    from tensorflow.keras.losses import MeanAbsoluteError
    mae = tensorflow.keras.losses.MeanAbsoluteError()
    model_mae = []
    list1_ok =[]
    list2_ok = []
    old_lists = [list1,list2]
    new_lists = [list1_ok,list2_ok]
    #creating list unifies by obs
    for j in [0,1]:
        temp_old_lists = old_lists[j]
        temp_new_lists = new_lists[j]
        for i in range(0,len(list1)-20, 21):
            temp = temp_old_lists[i:i+21]
            temp_new_lists.append(temp)
    # compute mse of each image
    for i in range(0,len(list1_ok)):
        model_mae.append(mae(list1_ok[i],list2_ok[i]).numpy())
    return model_mae
    
def apply_model():
    # Create the model
    model = Sequential()
    model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling3D(pool_size=(1,2,2)))
    model.add(BatchNormalization(center=True, scale=True))
    model.add(Dropout(0.5))
    model.add(Conv3D(64, kernel_size=(2,2,2), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling3D(pool_size=(1,2,2)))
    model.add(BatchNormalization(center=True, scale=True))
    model.add(Dropout(0.5))
    model.add(Conv3D(128, kernel_size=(2,2,2), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2)))
    model.add(BatchNormalization(center=True, scale=True))
    model.add(Dropout(0.5))
    model.add(Flatten())
    #model.add(Dense(48, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(36, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(24, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile the model
    model.compile(loss='binary_crossentropy',
                  optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.0001),
                  metrics=['accuracy'])
    return model



In [ ]:
########################
####### Model 1 ########
########################
# Fit data to model 1
x_train, y_train = create_data_for_run(1,'train')
model = apply_model()
model.build(input_shape=(None,5,25,431,1))
model.summary()
history_1 = model.fit(x_train, y_train, batch_size=60, epochs=90, verbose=1, validation_split=0.1)
del x_train, y_train

# cheking resalts
x_test, y_test = create_data_for_run(1, 'test')
predictions_1_model = model.predict(x_test)
predictions_1_model_cons = anomaly_decision(predictions_1_model, 1,'conservative')
predictions_1_model_liberal = anomaly_decision(predictions_1_model, 1,'liberal')

y_test = test_reduce_size(y_test)
Test_vector = y_test

In [ ]:
########################
####### Model 2 ########
########################

# Fit data to model 2
x_train, y_train = create_data_for_run(2,'train')
model = apply_model()
model.build(input_shape=(None,5,25,431,1))
model.summary()
history_2 = model.fit(x_train, y_train, batch_size=60, epochs=90, verbose=1, validation_split=0.1)
del x_train, y_train

# cheking resalts
x_test, y_test = create_data_for_run(2, 'test')
predictions_2_model = model.predict(x_test)
predictions_2_model_decision = anomaly_decision(predictions_2_model)

y_test = test_reduce_size(y_test)
Test_vector = y_test

In [ ]:
########################
####### Model 3 ########
########################
x_train, y_train = create_data_for_run(3, 'train')
audio_file_shape = (4,25,801)

def build_autoencoder(audio_file_shape, code_size):
    # The encoder
    encoder = Sequential()
    # encoder.add(InputLayer(None,4,25,801,1))
    encoder.add(Conv3D(32, kernel_size=(1,3,3), activation='relu', kernel_initializer='he_uniform'))
    encoder.add(Dropout(0.5))
    encoder.add(Conv3D(32, kernel_size=(1,2,2), activation='relu', kernel_initializer='he_uniform'))
    encoder.add(Dropout(0.5))
    encoder.add(GlobalAveragePooling3D())
    encoder.add(Dense(25*801))
    #encoder.add(Dense(code_size))
    # The decoder
    decoder = Sequential()
    decoder.add(InputLayer((25 * 801)))
    # decoder.add(Flatten())
    # decoder.add(Dense(25*801))
    decoder.add(Reshape((25, 801, 1)))
    return encoder, decoder

# Run Model
encoder, decoder = build_autoencoder((4, 25, 801), 1000)
inp = Input((4, 25, 801, 1))
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = Model(inp,reconstruction)
autoencoder.compile(optimizer='adamax', loss='mae')
print(autoencoder.summary())

history_3 = autoencoder.fit(x=x_train, y=y_train, epochs=20, validation_split=0.1)
MAE_model = history_3.history['loss']
MAE = MAE_model[-1]
del x_train, y_train

# Testing
x_test, y_test = create_data_for_run(3, 'test')
predictions_3_method = autoencoder.predict(x_test)

# compute the mae of each picture
predictions_3_method = np.reshape(predictions_3_method,[8400,25,801])
predictions_3_method.tolist()
y_test.tolist()
mae_test = compute_mae(predictions_3_method,y_test)